#### Задача 1 (10 баллов). 

Напишем мини-стратегическую игру, в которой игроку предлагается управлять городом (SimCity видели когда-нибудь?) У нас есть собственно класс "город", внутри города есть постройки разных типов, которые мы, как градоправитель, строим. Также у города есть атрибуты: уровень счастья жителей, уровень экологии, образования, миграции и так далее (на что вашей фантазии хватит). Постройки к этим атрибутам добавляют и прибавляют баллы. Можно завести тип постройки, которая на каждом ходе игры приносит в городской бюджет деньги (налоговая служба?.. заводы?). Постройки необходимо реализовать с помощью наследования, у некоторых построек могут быть свои методы, например, больница может устраивать медосмотры и вакцинации и поднимать этим уровень благосостояния жителей, а на заводе с рандомной вероятностью может случиться авария, которая понизит экологию. Не забудьте реализовать головной класс "игра", в котором будет метод play, можно в нем хранить и городской бюджет. 

Фантазия приветствуется: чем оригинальнее будет ваша игра, тем выше шанс, что я поставлю дополнительный балл. 

------------------------------------------------------------------------------------------------------

##### Механика игры:

Открывается главное меню с приветствием. Можно ввести 4 команды:
- посмотреть статус города (все его характеристики, кол-во построек);
- построить здание;
- собрать налоги с имеющихся зданий (возможно только при наличии налоговой службы);
- выйти из игры на совсем.

При выборе команды "построить здание" выводится список зданий, которые можно построить. После постройки здания характеристики города меняются некоторым образом. Некоторые особенности:
- каждая постройка приносит ущерб экологии, но разные постройки в разной степени;
- при постройке завода есть вероятность, что случится авария;
- при постройке полиции есть вероятность преступления, если народ в городе не очень довольный;
- при постройке "центра для досуга" в городе может произойти какое-то культурное событие, которое поднимет уровень счастья жителей;
- налоги можно собирать бесконечно раз...

In [71]:
from random import uniform

In [96]:
class Building:
    def __init__(self, name, cost, tax_income, happiness_boost, ecology_impact):
        self.name = name
        self.cost = cost
        self.tax_income = tax_income
        self.happiness_boost = happiness_boost
        self.ecology_impact = ecology_impact

    def operate(self, city):
        city.happiness += self.happiness_boost
        if city.happiness > 10:
            city.happiness = 10
        city.ecology -= self.ecology_impact
        if city.ecology < 0:
            city.ecology = 0


In [97]:
class Housing(Building):
    def __init__(self, name, cost, residents, tax_income, happiness_boost, ecology_impact):
        super().__init__(name, cost, tax_income, happiness_boost, ecology_impact) # импортируем конструктор родительского класса и добавляем новые атрибуты
        self.residents = residents
    
    def operate(self, city):
        super().operate(city) # вызываем метод operate из родительского класса Building
        city.population += self.residents
        city.migration += self.residents
        resident = 'жителей'
        x = self.residents % 10
        xx = self.residents % 100
        if not 11 <= xx < 15:
            if x == 1:
                resident = 'житель'
            elif x in {2, 3, 4}:
                resident = 'жителя'
        return f"{self.name} успешно построен, заселено {self.residents} {resident}.\nБюджет города: {city.budget} (-{self.cost}).\nУровень счастья жителей города повысился на {self.happiness_boost}\n"


In [98]:
# магазины, супермаркеты и т.п. 
class Commerce(Building):
    def operate(self, city):
        super().operate(city)
        return f"{self.name} успешно построено.\nБюджет города: {city.budget} (-{self.cost}).\nУровень счастья жителей города повысился на {self.happiness_boost}.\n"

In [99]:
class Police(Building):
    def __init__(self, name, cost, crime_reduction, happiness_boost, ecology_impact):
        super().__init__(name, cost, 0, happiness_boost, ecology_impact)
        self.crime_reduction = crime_reduction
    
    def operate(self, city):
        super().operate(city)
        city.crime -= self.crime_reduction
        if city.crime < 0:
            city.crime = 0
        return f"Уровень преступности снизился на {self.crime_reduction}.\nБюджет города: {city.budget} (-{self.cost}).\nУровень счастья жителей города повысился на {self.happiness_boost}.\n"
    
    def crime(self, city):
        # если жители недовольны и рандомное число меньше 0.4, то совершается преступление
        if city.happiness < 0.5 and uniform(0.0, 1.0) < 0.4:
            city.crime += 0.5
            return "В городе произоошло преступление. Уровень преступности повысился на 0.5.\n"
        return "В городе без происшествий, полиция патрулирует город.\n"

In [100]:
class Hospital(Building):
    def __init__(self, name, cost, health_boost, happiness_boost, ecology_impact):
        super().__init__(name, cost, 0, happiness_boost, ecology_impact)
        self.health_boost = health_boost

    def operate(self, city):
        super().operate(city)
        city.happiness += self.health_boost
        return f"Лечение населения успешно выполнено.\nБюджет города: {city.budget} (-{self.cost}).\nУровень счастья жителей города повысился на {self.happiness_boost}.\n"

In [101]:
class School(Building):
    def __init__(self, name, cost, education_boost, happiness_boost, ecology_impact):
        super().__init__(name, cost, 0, happiness_boost, ecology_impact)
        self.education_boost = education_boost
    
    def operate(self, city):
        super().operate(city)
        city.education += self.education_boost
        return f"{self.name} успешно построена.\nБюджет города: {city.budget} (-{self.cost}).\nУровень образования улучшился на {self.education_boost}.\n"

In [102]:
class Leisure(Building):
    def operate(self, city):
        super().operate(city)
        return f"{self.name} успешно построен.\nБюджет города: {city.budget} (-{self.cost}).\nУровень счастья жителей города поднялся на {self.happiness_boost}.\n"

    def event(self, city):
        if city.happiness > 0.1 and uniform(0.0, 1.0) < 0.5:
            city.happiness += 1.5
            city.budget += 100
            return f"Вот это сюрприз, в городе прошло культурное мероприятие!\nУровень счастья жителей поднялся на 1.5\n"

In [103]:
class Factory(Building):     
    def operate(self, city):
        super().operate(city)
        return f"{self.name} успешно построен.\nБюджет города: {city.budget} (-{self.cost}).\nУровень экологии снизился на 10.\n"
        
    def accident(self, city):
        if uniform(0.0, 1.0) < 0.35:
            city.ecology -= 20
            city.happiness -= 0.2
            city.crime += 0.1
            city.ecology -= 0.3
            city.population -= 10
            city.migration -= 10
            if city.population < 0 :
                city.population = 0
            return f'На заводе произошла авария. Уровень экологии снизился на 20.\n'
        return f'Завод работает в штатном режиме.\n'
        

In [104]:
class TaxOffice(Building):
    def __init__(self, name, cost, ecology_impact):
        super().__init__(name, cost, 0, 0, ecology_impact)
    
    def operate(self, city):
        super().operate(city)
        return f"{self.name} успешно построена.\nБюджет города: {city.budget} (-{self.cost}).\n"
    
    def collect_taxes(self, city):
        taxes = 0
        for building_list in city.buildings.values():
            for building in building_list:
                if isinstance(building, (Housing, Commerce, Factory, Leisure)):
                    taxes += building.tax_income
        city.budget += taxes
        return f"Собрано налогов на сумму {taxes}.\n"

In [105]:
class Park(Building):
    def __init__(self, name, cost, happiness_boost, ecology_impact):
        super().__init__(name, cost, 0, happiness_boost, ecology_impact)
    
    def operate(self, city):
        super().operate(city)
        return f"Парк построен! Жители очень довольны! Уровень счастья поднялся на {self.happiness_boost}.\n"

In [106]:
class City:
    def __init__(self, name, budget):
        self.name = name
        self.budget = budget
        self.population = 0
        self.happiness = 0 # 0-10
        self.ecology = 5 # 0-10
        self.education = 0 # 0-10
        self.migration = 0 # 0-10
        self.crime = 0 # 0-10
        self.buildings = {}
    
    def build_and_operate(self, building): # в один метод запихнула функцию добавления постройки в словарь и её действия на город
        if self.budget >= building.cost:
            if building.name in self.buildings:
                self.buildings[building.name].append(building)
            else:
                self.buildings[building.name] = [building]
            self.budget -= building.cost

            print(building.operate(self))
            return "f{building.name} добавлено в город.\n" 
        else:
            return f'Недостаточно средств бюджета для постройки здания.\n'
    
    def show_city_status(self):
        print(f'\nГород: {self.name}')
        print(f'Бюджет города {self.name}: {self.budget}')
        print(f'Население: {self.population}')
        print(f'Уровень счастья: {round(self.happiness, 2)}')
        print(f'Уровень экологии: {round(self.ecology, 2)}')
        print(f'Уровень образования: {round(self.education, 2)}')
        print(f'Уровень преступности: {round(self.education, 2)}')
        print(f'Уровень миграции: {round(self.education, 2)}')
        if self.buildings:
            print('Постройки города:')
            for building_name, buildings_list in self.buildings.items():
                print(f'\t{building_name}: {len(buildings_list)} шт.')
        else:
            print('В городе пока нет построек.\n')
        print('\n')

In [107]:
class Game:
    def __init__(self):
        self.city = City('Норт-Вилладж', 7000)
    
    def play(self):
        commands = [
            '1. Показать статус города',
            '2. Построить здание',
            '3. Собрать налоги с построек',
            '0. Выйти из игры',
            '\n'
        ]
        print('Привет! Добро пожаловать в симулятор города Норт-Вилладж!')
        
        while True:
            print('Выбери команду из списка ниже:\n')
            print('\n'.join(commands))
            command = input('Какое действие хочешь выполнить? Введи команду: ')

            if command == '1':
                self.city.show_city_status()
            elif command == '2':
                self.building_menu()
            elif command == '3':
                tax_office = self.find_building('Налоговая служба')
                if tax_office:
                    print(tax_office.collect_taxes(self.city))
                    print('---------------------------------------------')

                else:
                    print('В городе отсутствует налоговая служба. Нужно её построить!\n')
            elif command == '0':
                print(f'\nВыход из игры. До встречи!')
                break
            else:
                print('Введена неверная команда. Пожалуйста, выбери правильную команду.\n')

    def building_menu(self):
        choices = [
            '1. Жилой дом',
            '2. Школа',
            '3. Больница',
            '4. Полицейский участок',
            '5. Коммерческое здание',
            '6. Центр для досуга',
            '7. Завод',
            '8. Здание налоговой службы',
            '9. Парк',
            '0. Вернуться в главное меню\n'
        ]
        
        while True:
            print('Какое здание хочешь построить? Выбери из списка ниже:')
            print('\n'.join(choices))
            choice = input('Введи номер: ')

            if choice == '1':
                house = Housing('Жилой дом', 300, residents=15, tax_income=200, happiness_boost=1, ecology_impact=0.02)
                self.city.build_and_operate(house)
                print('---------------------------------------------')

            elif choice == '2':
                school = School('Школа', 500, education_boost=0.1, happiness_boost=0.9, ecology_impact=0.01)
                self.city.build_and_operate(school)
                print('---------------------------------------------')

            elif choice == '3':
                hospital = Hospital('Больница', 550, health_boost=0.1, happiness_boost=1.2, ecology_impact=0.02)
                self.city.build_and_operate(hospital)
                print('---------------------------------------------')

            elif choice == '4':
                police = Police('Полицейский участок', 400, crime_reduction=0.4, happiness_boost=1, ecology_impact=0.01)
                self.city.build_and_operate(police)
                print(police.crime(self.city))
                print('---------------------------------------------')

            elif choice == '5':
                commerce = Commerce('Магазин', 250, tax_income=300, happiness_boost=0.9, ecology_impact=0.03)
                self.city.build_and_operate(commerce)
                print('---------------------------------------------')

            elif choice == '6':
                leisure = Leisure('Центр для досуга', 250, tax_income=150, happiness_boost=1.2, ecology_impact=0.02)
                self.city.build_and_operate(leisure)
                print('---------------------------------------------')

            elif choice == '7':
                factory = Factory('Завод', 1000, tax_income=500, happiness_boost=1, ecology_impact=0.7)
                self.city.build_and_operate(factory)
                print(factory.accident(self.city))
                print('---------------------------------------------')

            elif choice == '8':
                tax_office = TaxOffice('Налоговая служба', 750, ecology_impact=0.01)
                self.city.build_and_operate(tax_office)
                print('---------------------------------------------')
            
            elif choice == '9':
                park = Park('Парк', 500, happiness_boost=1.8, ecology_impact=0.4)
                self.city.build_and_operate(park)
                print('---------------------------------------------')

            elif choice == '0':
                return # возвращение в главное меню
            else:
                print('Неверный выбор :( Попробуйте снова.\n')

    def find_building(self, building_name): # функция для поиска конкретного типа постройки (для отработки метода сбора налогов)
        if building_name in self.city.buildings and self.city.buildings[building_name]:
            return self.city.buildings[building_name][0]
        return None


In [108]:
game = Game()
game.play()

Привет! Добро пожаловать в симулятор города Норт-Вилладж!
Выбери команду из списка ниже:

1. Показать статус города
2. Построить здание
3. Собрать налоги с построек
0. Выйти из игры


Какое здание хочешь построить? Выбери из списка ниже:
1. Жилой дом
2. Школа
3. Больница
4. Полицейский участок
5. Коммерческое здание
6. Центр для досуга
7. Завод
8. Здание налоговой службы
9. Парк
0. Вернуться в главное меню

Жилой дом успешно построен, заселено 15 жителей.
Бюджет города: 6700 (-300).
Уровень счастья жителей города повысился на 1

---------------------------------------------
Какое здание хочешь построить? Выбери из списка ниже:
1. Жилой дом
2. Школа
3. Больница
4. Полицейский участок
5. Коммерческое здание
6. Центр для досуга
7. Завод
8. Здание налоговой службы
9. Парк
0. Вернуться в главное меню

Жилой дом успешно построен, заселено 15 жителей.
Бюджет города: 6400 (-300).
Уровень счастья жителей города повысился на 1

---------------------------------------------
Какое здание хочешь по